<a href="https://colab.research.google.com/github/PouyaZr/CNN_Cats_and_Dogs/blob/master/CNN_Cat_and_Dog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



### CAT and DOG --> CNN



In [ ]:
### Download dataset
!wget --no-check-certificate https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip

--2022-04-18 19:34:44--  https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip
Resolving download.microsoft.com (download.microsoft.com)... 173.223.108.114, 2600:1407:f800:4a5::e59, 2600:1407:f800:49b::e59
Connecting to download.microsoft.com (download.microsoft.com)|173.223.108.114|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 824894548 (787M) [application/octet-stream]
Saving to: ‘kagglecatsanddogs_3367a.zip’

kagglecatsanddogs_3 100%[===================>] 786.68M   103MB/s    in 7.9s    

2022-04-18 19:34:53 (99.2 MB/s) - ‘kagglecatsanddogs_3367a.zip’ saved [824894548/824894548]



In [ ]:
!ls

kagglecatsanddogs_3367a.zip  sample_data


In [ ]:
import zipfile

ZIP = zipfile.ZipFile('kagglecatsanddogs_3367a.zip', 'r')
ZIP.extractall()
ZIP.close()

In [23]:
!ls

'MSR-LA - 3467.docx'   PetImages  'readme[1].txt'   sample_data


In [22]:
!rm -rf kagglecatsanddogs_3367a.zip

In [ ]:
!mkdir /tmp/dataset
!mv PetImages /tmp/dataset

In [ ]:
!ls /tmp/dataset 

PetImages


In [ ]:
!rm -rf /tmp/dataset/train /tmp/dataset/test

In [ ]:
import os

try:
  ### Train, Test
  os.mkdir("/tmp/dataset/train")
  os.mkdir("/tmp/dataset/test")

  ### Train: Dog, Cat
  os.mkdir("/tmp/dataset/train/dogs")
  os.mkdir("/tmp/dataset/train/cats")

  ### Test: Dog, Cat
  os.mkdir("/tmp/dataset/test/dogs")
  os.mkdir("/tmp/dataset/test/cats")
except Exception as e:
  print(e)

In [ ]:
import random
from shutil import copyfile

In [ ]:
def split_data(SOURCE, TRAIN, TEST, SPLIT_SIZE):
  ### Reading the dataset
  dataset = os.listdir(SOURCE)      ### listdir returns lists of SOURCE

  ### Shuffling
  dataset_random = random.sample(dataset, len(dataset))
  # print(dataset_random[:10])

  # print(f"len: {len(dataset)}")

  ### Split size
  split_value = int(SPLIT_SIZE * len(dataset_random))

  # print(split_value)

  ### Iterate on dataset (TRAIN)
  for each_pic in dataset_random[:split_value]:
    if os.path.getsize(SOURCE+each_pic) != 0:
      copyfile(SOURCE+each_pic, TRAIN+each_pic)
  
  ### Iterate on dataset (TEST)
  for each_pic in dataset_random[split_value:]:
    if os.path.getsize(SOURCE+each_pic) != 0:
      copyfile(SOURCE+each_pic, TEST+each_pic)



dogs = "/tmp/dataset/PetImages/Dog/"
cats = "/tmp/dataset/PetImages/Cat/"
train_dogs = "/tmp/dataset/train/dogs/"
train_cats = "/tmp/dataset/train/cats/"
test_dogs = "/tmp/dataset/test/dogs/"
test_cats = "/tmp/dataset/test/cats/"


print(f"Dog Train: {len(os.listdir(train_dogs))}")
print(f"Dog Test: {len(os.listdir(test_dogs))}")
print(f"Cat Train: {len(os.listdir(train_cats))}")
print(f"Cat Test: {len(os.listdir(test_cats))}")
split_data(dogs, train_dogs, test_dogs, 0.8)
split_data(cats, train_cats, test_cats, 0.8)
print("*" * 50)
print(f"Dog Train: {len(os.listdir(train_dogs))}")
print(f"Dog Test: {len(os.listdir(test_dogs))}")
print(f"Cat Train: {len(os.listdir(train_cats))}")
print(f"Cat Test: {len(os.listdir(test_cats))}")

Dog Train: 0
Dog Test: 0
Cat Train: 0
Cat Test: 0
**************************************************
Dog Train: 9999
Dog Test: 2501
Cat Train: 9999
Cat Test: 2501


In [ ]:
print(f"Size of dataset: {os.path.getsize('/tmp/dataset/PetImages/Dog')}")
print(f"Size of dataset: {os.path.getsize('/tmp/dataset/PetImages/Cat')}")

Size of dataset: 270336
Size of dataset: 270336


In [ ]:
dataset = os.listdir("/tmp/dataset/PetImages/Cat")
print(dataset[0:10])

['11184.jpg', '8133.jpg', '401.jpg', '8498.jpg', '11251.jpg', '27.jpg', '9321.jpg', '10.jpg', '11581.jpg', '8451.jpg']


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
data_gen = ImageDataGenerator(rescale=1.0/255)

In [ ]:
train_gen = data_gen.flow_from_directory(directory='/tmp/dataset/train/',
                                         target_size=(150, 150),
                                         class_mode='binary',
                                         batch_size=32)

test_gen = data_gen.flow_from_directory(directory='/tmp/dataset/test/',
                                        target_size=(150, 150),
                                        class_mode='binary',
                                        batch_size=32)


Found 19996 images belonging to 2 classes.
Found 5002 images belonging to 2 classes.


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [ ]:
model = tf.keras.models.Sequential([
                                    ### First Layer
                                    Conv2D(filters=16, kernel_size=(3, 3), activation='relu', input_shape=(150, 150, 3)),
                                           MaxPooling2D(pool_size=(2, 2)),

                                    ### Second Layer
                                    Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
                                           MaxPooling2D(pool_size=(2, 2)),

                                    ### Third Layer
                                    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
                                           MaxPooling2D(pool_size=(2, 2)),

                                    ### Fourth Layer
                                    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
                                           MaxPooling2D(pool_size=(2, 2)),
                                    
                                    ### Fifth Layer
                                    Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
                                           MaxPooling2D(pool_size=(2, 2)),
                                    
                                    ### Flatten
                                    Flatten(),

                                    Dense(units=256, activation='relu'),
                                    Dense(units=1, activation='sigmoid')
                                                                          
])

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 148, 148, 16)      448       
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 74, 74, 16)       0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 72, 72, 32)        4640      
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 36, 36, 32)       0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 34, 34, 64)        18496     
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 17, 17, 64)      

In [ ]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
with tf.device('/device:GPU:0'):
  model.fit_generator(generator=train_gen,
                      steps_per_epoch=19996//32,
                      epochs=25,
                      validation_data=test_gen,
                      validation_steps=5002//32)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/25
482/624 [======================>.......] - ETA: 17s - loss: 0.6368 - accuracy: 0.6274

/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 32 bytes but only got 0. Skipping tag 270
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 5 bytes but only got 0. Skipping tag 271
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 272
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 282
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only

624/624 [==============================] - 109s 157ms/step - loss: 0.6194 - accuracy: 0.6475 - val_loss: 0.5445 - val_accuracy: 0.7242
Epoch 2/25
624/624 [==============================] - 92s 148ms/step - loss: 0.4963 - accuracy: 0.7654 - val_loss: 0.5009 - val_accuracy: 0.7564
Epoch 3/25
624/624 [==============================] - 92s 147ms/step - loss: 0.4201 - accuracy: 0.8072 - val_loss: 0.3993 - val_accuracy: 0.8213
Epoch 4/25
624/624 [==============================] - 93s 149ms/step - loss: 0.3580 - accuracy: 0.8432 - val_loss: 0.3591 - val_accuracy: 0.8476
Epoch 5/25
624/624 [==============================] - 93s 149ms/step - loss: 0.3065 - accuracy: 0.8671 - val_loss: 0.4113 - val_accuracy: 0.8243
Epoch 6/25
624/624 [==============================] - 93s 149ms/step - loss: 0.2658 - accuracy: 0.8887 - val_loss: 0.3051 - val_accuracy: 0.8754
Epoch 7/25
624/624 [==============================] - 93s 148ms/step - loss: 0.2281 - accuracy: 0.9052 - val_loss: 0.3158 - val_accuracy: 0.

In [ ]:
### The model has Overfitting

### Transfer Learning --> Cats and Dogs


  1) Instantiating our Model



In [2]:
import tensorflow as tf
import os
import zipfile

In [3]:
### InceptionV3
from tensorflow.keras.applications import InceptionV3

pre_trained_model = InceptionV3(input_shape=(150, 150, 3),
                                weights=None,
                                include_top=False)

In [4]:
!wget https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
-O /tmp/inception_v3_tf.h5

weights_address = "/tmp/inception_v3_tf.h5"

--2022-04-19 12:26:00--  https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.74.128, 209.85.145.128, 172.217.219.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.74.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87910968 (84M) [application/x-hdf]
Saving to: ‘/tmp/inception_v3_tf.h5’

/tmp/inception_v3_t 100%[===================>]  83.84M   220MB/s    in 0.4s    

2022-04-19 12:26:00 (220 MB/s) - ‘/tmp/inception_v3_tf.h5’ saved [87910968/87910968]



In [5]:
!ls /tmp

dap_multiplexer.ef37f9e8356f.root.log.INFO.20220419-122511.56
dap_multiplexer.INFO
debugger_qdadrfekc
inception_v3_tf.h5
initgoogle_syslog_dir.0
pyright-109-kPGaM43BX0oV
pyright-109-twi47l10lyY7
python-languageserver-cancellation


In [6]:
### Loading the weights that I have from another addresses
pre_trained_model.load_weights(weights_address)

In [7]:
### Freezing the weights that came from InceptionV3
for layer in pre_trained_model.layers:
  layer.trainable = False

In [8]:
# pre_trained_model.summary()

In [9]:
### Picking up last layer
last_layer = pre_trained_model.get_layer('mixed7')
last_output = last_layer.output

In [10]:
last_layer.output_shape

(None, 7, 7, 768)

2) Continuing to add the rest of the Model

In [11]:
from tensorflow.keras import layers
from tensorflow.keras import Model

In [12]:
myLayer = layers.Flatten()(last_output)

myLayer = layers.Dense(units=1024, activation='relu')(myLayer)

myLayer = layers.Dense(units=1, activation='sigmoid')(myLayer)

### Creating the ultimate model
model = Model(pre_trained_model.input, myLayer)

In [13]:
from tensorflow.keras.optimizers import RMSprop
model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer=RMSprop(learning_rate=0.0001))

In [14]:
### Download dataset
!wget --no-check-certificate https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip

--2022-04-19 12:26:05--  https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip
Resolving download.microsoft.com (download.microsoft.com)... 23.11.196.107, 2600:1407:f800:4a5::e59, 2600:1407:f800:49b::e59
Connecting to download.microsoft.com (download.microsoft.com)|23.11.196.107|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 824894548 (787M) [application/octet-stream]
Saving to: ‘kagglecatsanddogs_3367a.zip’

kagglecatsanddogs_3 100%[===================>] 786.68M   181MB/s    in 4.4s    

2022-04-19 12:26:09 (178 MB/s) - ‘kagglecatsanddogs_3367a.zip’ saved [824894548/824894548]



In [15]:
ZIP = zipfile.ZipFile('kagglecatsanddogs_3367a.zip', 'r')
ZIP.extractall()
ZIP.close()

In [16]:
!mkdir /tmp/dataset
!mv PetImages /tmp/dataset

In [17]:
try:
  ### Train, Test
  os.mkdir("/tmp/dataset/train")
  os.mkdir("/tmp/dataset/test")

  ### Train: Dog, Cat
  os.mkdir("/tmp/dataset/train/dogs")
  os.mkdir("/tmp/dataset/train/cats")

  ### Test: Dog, Cat
  os.mkdir("/tmp/dataset/test/dogs")
  os.mkdir("/tmp/dataset/test/cats")
except Exception as e:
  print(e)

In [18]:
import random
from shutil import copyfile

In [19]:
def split_data(SOURCE, TRAIN, TEST, SPLIT_SIZE):
  ### Reading the dataset
  dataset = os.listdir(SOURCE)      ### listdir returns lists of SOURCE

  ### Shuffling
  dataset_random = random.sample(dataset, len(dataset))
  # print(dataset_random[:10])

  # print(f"len: {len(dataset)}")

  ### Split size
  split_value = int(SPLIT_SIZE * len(dataset_random))

  # print(split_value)

  ### Iterate on dataset (TRAIN)
  for each_pic in dataset_random[:split_value]:
    if os.path.getsize(SOURCE+each_pic) != 0:
      copyfile(SOURCE+each_pic, TRAIN+each_pic)
  
  ### Iterate on dataset (TEST)
  for each_pic in dataset_random[split_value:]:
    if os.path.getsize(SOURCE+each_pic) != 0:
      copyfile(SOURCE+each_pic, TEST+each_pic)



dogs = "/tmp/dataset/PetImages/Dog/"
cats = "/tmp/dataset/PetImages/Cat/"
train_dogs = "/tmp/dataset/train/dogs/"
train_cats = "/tmp/dataset/train/cats/"
test_dogs = "/tmp/dataset/test/dogs/"
test_cats = "/tmp/dataset/test/cats/"


print(f"Dog Train: {len(os.listdir(train_dogs))}")
print(f"Dog Test: {len(os.listdir(test_dogs))}")
print(f"Cat Train: {len(os.listdir(train_cats))}")
print(f"Cat Test: {len(os.listdir(test_cats))}")
split_data(dogs, train_dogs, test_dogs, 0.8)
split_data(cats, train_cats, test_cats, 0.8)
print("*" * 50)
print(f"Dog Train: {len(os.listdir(train_dogs))}")
print(f"Dog Test: {len(os.listdir(test_dogs))}")
print(f"Cat Train: {len(os.listdir(train_cats))}")
print(f"Cat Test: {len(os.listdir(test_cats))}")

Dog Train: 0
Dog Test: 0
Cat Train: 0
Cat Test: 0
**************************************************
Dog Train: 10000
Dog Test: 2500
Cat Train: 9999
Cat Test: 2501


In [20]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [21]:
train_gen = ImageDataGenerator(rescale=1.0/255,
                               rotation_range=20,
                               width_shift_range=0.2,
                               height_shift_range=0.2,
                               shear_range=0.2,
                               zoom_range=0.2,
                               fill_mode='nearest')

In [22]:
test_gen = ImageDataGenerator(rescale=1.0/255)

In [23]:
train_generator = train_gen.flow_from_directory(directory="/tmp/dataset/train",
                                               target_size=(150, 150),
                                               class_mode='binary',
                                               batch_size=20)

test_generator = test_gen.flow_from_directory(directory="/tmp/dataset/test",
                                              target_size=(150, 150),
                                              class_mode='binary',
                                              batch_size=20)

Found 19997 images belonging to 2 classes.
Found 5001 images belonging to 2 classes.


In [24]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [25]:
with tf.device("/device:GPU:0"):
  H = model.fit(train_generator,
            epochs=20,
            steps_per_epoch=19996//20,
            validation_data=test_generator,
            validation_steps=5002//20)

Epoch 1/20
438/999 [============>.................] - ETA: 1:54 - loss: 0.2088 - accuracy: 0.9190

/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 32 bytes but only got 0. Skipping tag 270
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 5 bytes but only got 0. Skipping tag 271
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 272
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 282
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only

999/999 [==============================] - 241s 227ms/step - loss: 0.1862 - accuracy: 0.9297 - val_loss: 0.1019 - val_accuracy: 0.9696
Epoch 2/20
999/999 [==============================] - 226s 227ms/step - loss: 0.1480 - accuracy: 0.9499 - val_loss: 0.1288 - val_accuracy: 0.9608
Epoch 3/20
999/999 [==============================] - 225s 225ms/step - loss: 0.1446 - accuracy: 0.9524 - val_loss: 0.0938 - val_accuracy: 0.9710
Epoch 4/20
999/999 [==============================] - 230s 230ms/step - loss: 0.1353 - accuracy: 0.9540 - val_loss: 0.0924 - val_accuracy: 0.9720
Epoch 5/20
999/999 [==============================] - 230s 230ms/step - loss: 0.1200 - accuracy: 0.9605 - val_loss: 0.0975 - val_accuracy: 0.9718
Epoch 6/20
999/999 [==============================] - 232s 232ms/step - loss: 0.1249 - accuracy: 0.9590 - val_loss: 0.0938 - val_accuracy: 0.9720
Epoch 7/20
999/999 [==============================] - 230s 230ms/step - loss: 0.1098 - accuracy: 0.9631 - val_loss: 0.0942 - val_accura